In [6]:
!pip install google-search-results

In [7]:
# Import things that are needed generically
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

# Initialize the LLM to use for the agent.

llm = ChatOpenAI(temperature=0)

# Completely New Tools - String Input and Output
# The simplest tools accept a single query string and return a string output. If your tool function requires multiple arguments, you might want to skip down to the StructuredTool section below.

# There are two ways to do this: either by using the Tool dataclass, or by subclassing the BaseTool class.

# Tool dataclass
# The 'Tool' dataclass wraps functions that accept a single string input and returns a string output.

# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]


/Users/ianderrington/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [15]:

#     /Users/wfh/code/lc/lckg/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
#       warnings.warn(


# You can also define a custom args_schema to provide more information about inputs.

from pydantic import Field
from pydantic.v1 import BaseModel

class CalculatorInput(BaseModel):
    question: str = Field(...,
        description="The process ID (PID) of the process to monitor.")


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        # args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)

    
   

TypeError: Object of type 'FieldInfo' is not JSON serializable

In [13]:
 
#     > Entering new AgentExecutor chain...
#     I need to find out Leo DiCaprio's girlfriend's name and her age
#     Action: Search
#     Action Input: "Leo DiCaprio girlfriend"
#     Observation: After rumours of a romance with Gigi Hadid, the Oscar winner has seemingly moved on. First being linked to the television personality in September 2022, it appears as if his "age bracket" has moved up. This follows his rumoured relationship with mere 19-year-old Eden Polani.
#     Thought:I still need to find out his current girlfriend's name and age
#     Action: Search
#     Action Input: "Leo DiCaprio current girlfriend"
#     Observation: Just Jared on Instagram: “Leonardo DiCaprio & girlfriend Camila Morrone couple up for a lunch date!
#     Thought:Now that I know his girlfriend's name is Camila Morrone, I need to find her current age
#     Action: Search
#     Action Input: "Camila Morrone age"
#     Observation: 25 years
#     Thought:Now that I have her age, I need to calculate her age raised to the 0.43 power
#     Action: Calculator
#     Action Input: 25^(0.43)
    
#     > Entering new LLMMathChain chain...
#     25^(0.43)```text
#     25**(0.43)
#     ```
#     ...numexpr.evaluate("25**(0.43)")...
    
#     Answer: 3.991298452658078
#     > Finished chain.
    
#     Observation: Answer: 3.991298452658078
#     Thought:I now know the final answer
#     Final Answer: Camila Morrone's current age raised to the 0.43 power is approximately 3.99.
    
#     > Finished chain.





#     "Camila Morrone's current age raised to the 0.43 power is approximately 3.99."


# Subclassing the BaseTool class
# You can also directly subclass BaseTool. This is useful if you want more control over the instance variables or if you want to propagate callbacks to nested chains or other tools.

from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return search.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return llm_math_chain.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

tools = [CustomSearchTool(), CustomCalculatorTool()]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)

TypeError: Object of type 'FieldInfo' is not JSON serializable